# Credit Risk Resampling Techniques

In [49]:
import warnings
warnings.filterwarnings('ignore')

In [50]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split #to for the train_test_split function

# Read the CSV into DataFrame

In [51]:
# Load the data
#using pandas to read in csv file
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.tail() #shows there are over 70,000 rows

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk
77535,15600.0,9.742,mortgage,72300,0.585062,9,2,42300,high_risk


In [52]:
df= pd.get_dummies(df, columns = ['homeowner'])
#wouldn't let me scale without having all columns in rows display floats/integers
#.get_dummies() converts categorical variables into dummy/indicator variables.

# Split the Data into Training and Testing

In [53]:
# Create our features
X = df.drop(columns="loan_status") #x (features) is all columns without loan_status / x is what we will use to predict

# Create our target
y = df["loan_status"] #y equals this specific column (target)/ what we want to predict
#dealing with 2 classes, high_risk and low_risk

In [54]:
X.shape #data set has 10 columns (doesn't include 'loan status')

(77536, 10)

In [55]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [56]:
# Check the balance of our target values
y.value_counts()
#low_risk is marjoity
#high_risk is minority

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [57]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, #column created above
                                                    y, #column created above
                                                    random_state=1, 
                                                    stratify=y)

#using this data to train and test to see if predictions of actual data is accurate (if so, we can predict new data)

#around 70% of the data is going to training,and around 30% going to testing

X_train.shape #as you can see, 58152 out of 77536 rows is around 75% of data

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [58]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
#Scaling is used so that one column doesn't have too much bias
ds = StandardScaler()

In [59]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = ds.fit(X_train) #using it on X_train (training dataset)


In [60]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [61]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [62]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = classifier.predict(X_test) #predicts y using (the remaining 25% of data)

balanced_accuracy_score(y_test, y_pred) #compares y_test to y_predictions
#use balanced accuracy score to deal with imbalanced datasets. 

0.9543211898288821

In [63]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [64]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred, digits=4))
#this builds a classification report based on metrics used with imbalanced dataset.

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8599    0.9136    0.9950    0.8860    0.9535    0.9017       625
   low_risk     0.9971    0.9950    0.9136    0.9961    0.9535    0.9165     18759

avg / total     0.9927    0.9924    0.9162    0.9925    0.9535    0.9160     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [65]:
# Resample the training data with the RandomOversampler
#Oversampling adds samples to minority class
from imblearn.over_sampling import RandomOverSampler
#only resample training data
#Says to use random state of 1 for each alogrithm
ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train) #fits X_train_scaled and y_train
#using scaled data to make sure no bias

# View the count of target classes with Counter
Counter(y_resampled) #now there are the same amount for both

Counter({'low_risk': 56277, 'high_risk': 56277})

In [66]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [67]:
# Calculated the balanced accuracy score
y_pred_over = classifier.predict(X_test_scaled) #predicts y using the X_text_scaled (the remaining 25% of data (scaled))
balanced_accuracy_score(y_test, y_pred_over) #compares y_test to y_predictions

0.9946414201183431

In [68]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_over)

array([[  622,     3],
       [  111, 18648]])

In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_over, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8486    0.9952    0.9941    0.9161    0.9946    0.9894       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9946    0.9892     18759

avg / total     0.9950    0.9941    0.9952    0.9943    0.9946    0.9892     19384



### SMOTE Oversampling

In [70]:
from imblearn.over_sampling import SMOTE #allows us to use SMOTE

In [71]:
# Resample the training data with SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [72]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [73]:
# Calculated the balanced accuracy score
y_pred_smote = classifier.predict(X_test_scaled) 
balanced_accuracy_score(y_test, y_pred_smote) #compares y_test to y_predictions

0.9946680739911509

In [74]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

array([[  622,     3],
       [  110, 18649]])

In [75]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote, digits=4))
#slighty better than OverSampling based on confusion matrix

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8497    0.9952    0.9941    0.9167    0.9947    0.9895       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9947    0.9893     18759

avg / total     0.9950    0.9942    0.9952    0.9944    0.9947    0.9893     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [76]:
from imblearn.under_sampling import ClusterCentroids #allows us to use ClusterCentroids

In [77]:
# Resample the data using the ClusterCentroids resampler
#decreases size of majority class
cc = ClusterCentroids(random_state=1, sampling_strategy=1.0)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [78]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [79]:
# Calculate the balanced accuracy score
y_pred_cc = classifier.predict(X_test_scaled) 
balanced_accuracy_score(y_test, y_pred_cc)

0.9932813049736127

In [80]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

array([[  620,     5],
       [  102, 18657]])

In [81]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8587    0.9920    0.9946    0.9206    0.9933    0.9864       625
   low_risk     0.9997    0.9946    0.9920    0.9971    0.9933    0.9869     18759

avg / total     0.9952    0.9945    0.9921    0.9947    0.9933    0.9868     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [82]:
from imblearn.combine import SMOTEENN #allows us to use SMOTEEN

In [83]:
# Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1, sampling_strategy=1.0)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55428, 'low_risk': 55906})

In [84]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [85]:
# Calculate the balanced accuracy score
y_pred_smoteen = classifier.predict(X_test_scaled) 
balanced_accuracy_score(y_test, y_pred_smoteen)

0.9946680739911509

In [86]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smoteen)

array([[  622,     3],
       [  110, 18649]])

In [87]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smoteen, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8497    0.9952    0.9941    0.9167    0.9947    0.9895       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9947    0.9893     18759

avg / total     0.9950    0.9942    0.9952    0.9944    0.9947    0.9893     19384



Accuracy scores - 
    Logistic Regression: 0.9543211898288821
   
    OverSampling: 0.9946414201183431
    
    SMOTE Oversampling:0.9946680739911509
    
    UnderSampling CC:0.9932813049736127

    SMOTEENN: 0.9946680739911509
    

# Final Questions

1. Which model had the best balanced accuracy score?

   The SMOTE model and SMOTEENN model have the same scores, but are the highest scores. Both had an accuracy score of
   0.9946680739911509

2. Which model had the best recall score?

    The Undersampling Cluster Centroids model had the best recall score, which was .9945
    
    (Most models had a very good recall score, over .98)

3. Which model had the best geometric mean score?

    Similarly with accuracy score, the SMOTE model and SMOTEEN model have the same and highest geometric mean score.
    Both had a geeometic score of .9947
